## Q1
std of March 2023 predictions: 6.24

Solution:

### Model Training, Validation (again, just to make sure we can reproduce the previous results)

In [2]:
from model_training import train_pipeline, validation_pipeline

train_pipeline()

Fetching yellow taxi trip data for 2023-01
Dataset contains 3066766 records and 19 columns
98.1220282212598% of records are kept after outlier removal
fit-transforming the features
515 features will be used
Training regressor of type LR
RMSE: 7.649261929201487


In [3]:
import pickle

with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

validation_pipeline(dv, model)

Fetching yellow taxi trip data for 2023-02
Dataset contains 2913955 records and 19 columns
98.00944077722545% of records are kept after outlier removal
transforming the features
515 features will be used
RMSE: 7.811819793542861


### Prediction

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
import pandas as pd

categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
y_pred.std()

np.float64(6.2474855298130505)

## Q2
Size of the results file is 66M.

Solution:

In [9]:
df = pd.DataFrame({'duration': y_pred})

In [10]:
year = 2023
month = 3
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [11]:
df.to_parquet(
    'predictions_{}-{:02d}.parquet'.format(year, month),
    engine='pyarrow',
    compression=None,
    index=False
)

In [12]:
! ls -lh predictions_2023-03.parquet

-rw-rw-r-- 1 onur onur 66M Jun 10 23:55 predictions_2023-03.parquet


## Q3: Creating the scoring script

the required command is : `jupyter nbconvert --to script <file_name>.ipynb` where, in this case, `file_name` is `04_Homework.ipynb`

## Q4: Virtual environment. Hash for Scikit-learn

## Q5: Parameterize the script

The mean prediction for April 2023 is 14.29

Solution:

In [1]:
from model_prediction import TaxiDurationPredictor

td_predictor = TaxiDurationPredictor('./')
td_predictor.run(2023, 4)

reading from cloud: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-04.parquet
predicted mean duration: 14.292284022509667


## Q6

Run the script with docker. What's the mean predicted duration for May 2023?

Answer: 0.1917
For the solution, see [web-service-homework](web-service-homework) directory.
